In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np
import os 
import requests
import pickle
import sys

In [2]:
def parse_bet(soup):
    all_elements = soup.find_all('div', class_="flex flex-col px-3 text-sm max-mm:px-0")
    all_record = list()
    for i in all_elements:
        all_div = i.find("div")
        all_div_list = all_div.find_all("div", class_="eventRow")
        date_res = ""
        for j in all_div_list:
            bet_record = list()
            date_div = j.find("div", class_="border-l")
            res_div = j.find("div", class_="border-b")
            if date_div.find("div", class_="flex items-center justify-start w-full pl-2 bg-gray-light border-black-borders") != None:
                date_res = date_div.find("div", class_="flex items-center justify-start w-full pl-2 bg-gray-light border-black-borders").find("div").contents[0]
            bet_record.append(date_res)
            res_list = res_div.find("div").find("a").find("div", class_="flex items-center gap-1 my-1 align-center w-[100%]").find("div").find("div").find_all("a")
            for k in res_list:
                bet_record.append(k['title'])
            bet_list = res_div.find("div").find_all("div", class_="flex-center flex-col gap-1 pt-1 pb-1 border-black-borders min-w-[60px]")
            for b in bet_list:
                bet_record.append(b.find("p").contents[0])
            all_record.append(bet_record)
    return all_record

In [ ]:
all_record = list()
for i in range(25):
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled") 
    options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
    options.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Chrome(r'C:\Users\User\Documents\ntu\crawler\chromedriver.exe', options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    page_url = "https://www.oddsportal.com/basketball/usa/nba-2022-2023/results/#/page/{}/".format(str(i+1))
    driver.get(page_url)
    time.sleep(15)
    html = driver.page_source
    nba_soup = BeautifulSoup(html, "html.parser")
    res = parse_bet(nba_soup)
    all_record.extend(res)
    driver.close()
    

<ipython-input-4-0cefe1eb3597>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\User\Documents\ntu\crawler\chromedriver.exe', options=options)


In [15]:
df = pd.DataFrame(all_record, columns =['Date', 'team1', 'team2', 'team1_odds', 'team2_odds']) 

In [16]:
df.to_csv('bet_2223.csv', index=False)  

In [17]:
df

,Date,team1,team2,team1_odds,team2_odds
0,12 Oct 2020 - Play Offs,Miami Heat,Los Angeles Lakers,3.02,1.41
1,10 Oct 2020 - Play Offs,Los Angeles Lakers,Miami Heat,1.31,3.62
2,07 Oct 2020 - Play Offs,Miami Heat,Los Angeles Lakers,3.78,1.29
3,05 Oct 2020 - Play Offs,Miami Heat,Los Angeles Lakers,4.41,1.23
4,03 Oct 2020 - Play Offs,Los Angeles Lakers,Miami Heat,1.17,5.48
...,...,...,...,...,...
1240,06 Oct 2019 - Pre-season,San Antonio Spurs,Orlando Magic,1.35,3.22
1241,06 Oct 2019 - Pre-season,Golden State Warriors,Los Angeles Lakers,2.49,1.54
1242,05 Oct 2019 - Pre-season,Indiana Pacers,Sacramento Kings,2.46,1.54
1243,04 Oct 2019 - Pre-season,Sacramento Kings,Indiana Pacers,1.48,2.66
